In [1]:
# Installer ViennaRNA
!pip install ViennaRNA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 28.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import pandas as pd
import RNA

In [3]:
# Monter Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Charger le CSV existant
csv_path = "/content/drive/MyDrive/stage PFE ML classifieur /stage PFE selected_non_conserved_miRNAs2.csv"
df = pd.read_csv(csv_path)

In [8]:
print(df.columns)

Index(['code de miRNA', 'nombre of species', 'séquence de miRNA',
       'miRNA length', 'GC content (%)', 'Seed GC content (positions 2-8)',
       'Thermodynamic stability (deltaG)', 'overall_pairing_probability',
       'espèce'],
      dtype='object')


In [6]:
# Fonction pour calculer le GC%
def calculate_gc(seq):
    gc = seq.count('G') + seq.count('C')
    return round((gc / len(seq)) * 100, 2) if seq else 0

# Fonction pour le GC% de la graine (positions 2-8)
def calculate_seed_gc(seq):
    seed = seq[1:8] if len(seq) >= 8 else ''
    return calculate_gc(seed) if seed else 0



# Configuration spécifique pour les plantes
PLANT_TEMP = 25.0  # Température standard pour les plantes
MIN_LENGTH = 18    # Longueur minimale typique des miRNA végétaux

def calculate_deltaG_plant(seq):
    """Calcule le deltaG avec paramètres adaptés aux plantes"""
    try:
        # Nettoyage et validation de la séquence
        clean_seq = seq.upper().replace('T', 'U').replace('-', '')

        # Filtrage des séquences non valides
        if len(clean_seq) < MIN_LENGTH or any(c not in 'ACGU' for c in clean_seq):
            return None

        # Configuration du modèle thermodynamique
        md = RNA.md()
        md.temperature = PLANT_TEMP    # Adapté aux conditions végétales
        md.dangles = 2                # Modèle de dangles amélioré
        md.noLP = 1                   # Pas de paires isolées
        md.logML = 1                   # Correction du volume exclu

        # Création de l'objet de calcul
        fc = RNA.fold_compound(clean_seq, md)

        # Calcul de la structure optimale
        structure, mfe = fc.mfe()

        # Validation du résultat
        if not -11.0 <= mfe <= -0.1:  # Plage attendue pour les végétaux
            return None

        return round(mfe, 2)

    except Exception as e:
        print(f"Erreur sur {seq[:15]}...: {str(e)}")
        return None

# Application au DataFrame
df['Thermodynamic stability (deltaG)'] = df['séquence de miRNA'].apply(calculate_deltaG_plant)

# Filtrage des résultats invalides
df = df[df['Thermodynamic stability (deltaG)'].notna()]



print("Calcul deltaG végétal réussi avec ViennaRNA!")
print(f"{len(df)} séquences valides (-11 ≤ deltaG ≤ -0.1)")


df['miRNA length'] = df['séquence de miRNA'].apply(len)
df['GC content (%)'] = df['séquence de miRNA'].apply(calculate_gc)
df['Seed GC content (positions 2-8)'] = df['séquence de miRNA'].apply(calculate_seed_gc)
df['Thermodynamic stability (deltaG)'] = df['séquence de miRNA'].apply(calculate_deltaG_plant)


# Nouvel ordre des colonnes
new_order = [
    'code de miRNA',
    'nombre of species',
    'séquence de miRNA',
    'miRNA length',
    'GC content (%)',
    'Seed GC content (positions 2-8)',
    'Thermodynamic stability (deltaG)',  # Nom modifié
    'espèce'
]

# Sauvegarde
df[new_order].to_csv(csv_path, index=False)

print("CSV update completed successfully!")
print(f"New columns added: {list(new_order[3:-1])}")



KeyError: 'séquence de miRNA'

In [ ]:
def calculate_overall_pairing(seq):
    """Calcule la probabilité d'appariement globale d'une séquence d'ARN (normalisée entre 0 et 1)"""
    try:
        # Nettoyage et validation
        clean_seq = seq.upper().replace('T', 'U').replace('-', '')
        valid_chars = {'A', 'C', 'G', 'U'}

        if not all(c in valid_chars for c in clean_seq):
            return None

        if len(clean_seq) < 2:
            return None

        # Calcul de la fonction de partition
        fc = RNA.fold_compound(clean_seq)
        fc.pf()  # Compute partition function
        bpp_matrix = fc.bpp()  # Get base pair probability matrix

        # Calcul de la probabilité d'appariement globale
        total_prob = 0.0
        for i in range(len(clean_seq)):
            for j in range(i + 1, len(clean_seq)):
                total_prob += bpp_matrix[i][j]

        # Normalisation pour obtenir une probabilité moyenne par paire
        num_pairs = len(clean_seq) * (len(clean_seq) - 1) / 2  # Nombre de paires possibles
        avg_prob = total_prob / num_pairs if num_pairs > 0 else 0  # Probabilité moyenne par paire

        return round(avg_prob, 3)

    except Exception as e:
        print(f"Erreur avec {seq[:15]}...: {str(e)}")
        return None
# Ajouter la nouvelle colonne
df['overall_pairing_probability'] = df['séquence de miRNA'].apply(calculate_overall_pairing)

# Réorganiser les colonnes - Vérifier et corriger les noms de colonnes
cols = [
    'code de miRNA',
    'nombre of species',
    'séquence de miRNA',
    'miRNA length',  # Vérifier ce nom de colonne dans votre CSV
    'GC content (%)',
    'Seed GC content (positions 2-8)',  # Vérifier ce nom de colonne dans votre CSV
    'Thermodynamic stability (deltaG)',  # Vérifier ce nom de colonne dans votre CSV
    'overall_pairing_probability',
    'espèce'
]
# Sauvegarder
df[cols].to_csv(csv_path, index=False)

print("Colonne de probabilité globale ajoutée avec succès !")
print("Statistiques descriptives :")
print(df['overall_pairing_probability'].describe())

Colonne de probabilité globale ajoutée avec succès !
Statistiques descriptives :
count    787.000000
mean       0.015663
std        0.005989
min        0.002000
25%        0.012000
50%        0.015000
75%        0.019500
max        0.034000
Name: overall_pairing_probability, dtype: float64


In [ ]:
import pandas as pd
from google.colab import drive

# Monter Google Drive pour accéder aux fichiers
drive.mount('/content/drive')

# Chemin du fichier CSV sur Google Drive
file_path = "/content/drive/MyDrive/stage PFE ML classifieur /stage PFE selected_non_conserved_miRNAs2.csv"  # Assurez-vous que le chemin est correct

# Lecture du fichier CSV
df = pd.read_csv(file_path)

def calculate_z_curve(seq):
    """Calcule les paramètres Z-curve pour chaque position de la séquence"""
    try:
        # Nettoyage comme dans calculate_overall_pairing
        clean_seq = seq.upper().replace('T', 'U').replace('-', '')
        valid_chars = {'A', 'C', 'G', 'U'}

        if not all(c in valid_chars for c in clean_seq):
            return [None] * (3 * len(clean_seq))

        x_total = 0
        y_total = 0
        z_total = 0
        z_params = []

        for idx, nt in enumerate(clean_seq, 1):
            # Calcul des composantes pour chaque nucléotide
            if nt == 'A':
                dx, dy, dz = 1, 1, 1
            elif nt == 'C':
                dx, dy, dz = -1, 1, -1
            elif nt == 'G':
                dx, dy, dz = 1, -1, -1
            elif nt == 'U':
                dx, dy, dz = -1, -1, 1
            else:
                dx, dy, dz = 0, 0, 0

            # Mise à jour des sommes cumulatives
            x_total += dx
            y_total += dy
            z_total += dz

            # Ajout des valeurs pour cette position
            z_params.extend([x_total, y_total, z_total])

        return z_params

    except Exception as e:
        print(f"Erreur Z-curve avec {seq[:15]}...: {str(e)}")
        return [None] * (3 * len(seq))

# Ajouter la colonne de probabilité globale
df['overall_pairing_probability'] = df['séquence de miRNA'].apply(calculate_overall_pairing)

# Générer les colonnes Z-curve
z_curve_data = df['séquence de miRNA'].apply(calculate_z_curve)

# Déterminer le nombre maximal de colonnes générées
max_columns = max(len(x) for x in z_curve_data)

# Normaliser les listes pour éviter le décalage
z_curve_data = z_curve_data.apply(lambda x: x + [None] * (max_columns - len(x)))

# Créer le DataFrame Z-curve
z_curve_columns = [f'x_{i//3+1}' if i % 3 == 0 else f'y_{i//3+1}' if i % 3 == 1 else f'z_{i//3+1}' for i in range(max_columns)]
z_curve_df = pd.DataFrame(z_curve_data.tolist(), columns=z_curve_columns)

# Fusionner avec le DataFrame principal
df = pd.concat([df, z_curve_df], axis=1)

# Supprimer les colonnes vides (qui ne contiennent aucune valeur)
df = df.dropna(axis=1, how='all') # Changed df_combined to df

# Supprimer les colonnes spécifiques
columns_to_remove = ['x_20', 'y_20', 'z_20', 'x_21', 'y_21', 'z_21', 'x_22', 'y_22', 'z_22',
                     'x_23', 'y_23', 'z_23', 'x_24', 'y_24', 'z_24', 'x_25', 'y_25', 'z_25']
df = df.drop(columns=[col for col in columns_to_remove if col in df.columns]) # Changed df_combined to df

# Sauvegarder toutes les colonnes dans le fichier CSV mis à jour
csv_output_path = "/content/drive/MyDrive/stage PFE ML classifieur /stage PFE selected_non_conserved_miRNAs1.csv"
df.to_csv(csv_output_path, index=False)

print("Colonnes Z-curve ajoutées avec succès !")
print(f"Nombre total de colonnes : {len(df.columns)}")
print("Exemple de nouvelles colonnes :", df.columns[-6:].tolist())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colonnes Z-curve ajoutées avec succès !
Nombre total de colonnes : 66
Exemple de nouvelles colonnes : ['x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19']


cette cellule est réalisé pour le fichier test final


In [ ]:
# Supprimer les colonnes vides (qui ne contiennent aucune valeur)
df = df.dropna(axis=1, how='all') # Changed df_combined to df

# Supprimer les colonnes spécifiques
columns_to_remove = ['x_20', 'y_20', 'z_20', 'x_21', 'y_21', 'z_21', 'x_22', 'y_22', 'z_22',
                     'x_23', 'y_23', 'z_23', 'x_24', 'y_24', 'z_24', 'x_25', 'y_25', 'z_25']
df = df.drop(columns=[col for col in columns_to_remove if col in df.columns]) # Changed df_combined to df


In [ ]:
print(df.columns)

Index(['code de miRNA', 'nombre of species', 'séquence de miRNA',
       'miRNA length', 'GC content (%)', 'Seed GC content (positions 2-8)',
       'Thermodynamic stability (deltaG)', 'overall_pairing_probability',
       'espèce', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2', 'x_3', 'y_3', 'z_3',
       'x_4', 'y_4', 'z_4', 'x_5', 'y_5', 'z_5', 'x_6', 'y_6', 'z_6', 'x_7',
       'y_7', 'z_7', 'x_8', 'y_8', 'z_8', 'x_9', 'y_9', 'z_9', 'x_10', 'y_10',
       'z_10', 'x_11', 'y_11', 'z_11', 'x_12', 'y_12', 'z_12', 'x_13', 'y_13',
       'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15', 'x_16', 'y_16',
       'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19',
       'z_19'],
      dtype='object')
